In [ ]:
# 各種ライブラリをインポート

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
import matplotlib.pyplot as plt

import datetime

In [ ]:
# GPUの利用可否

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

In [ ]:
# data-loader

In [ ]:
image_size = 224
degress1 = 30
degress2 = 60

data_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degress1),
    transforms.RandomRotation(degress2),
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomErasing(value='random')
])

In [ ]:
# dir_loader

In [ ]:
batch_size = 50
num_workers = 10

#trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
#testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
#testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

trainset = torchvision.datasets.ImageFolder(root='./dataset/tiny-imagenet-200/train', transform=data_transform, target_transform=None, is_valid_file=None)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
testset = torchvision.datasets.ImageFolder(root='./dataset/tiny-imagenet-200/test', transform=data_transform, target_transform=None, is_valid_file=None)
testloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [ ]:
flatten_features = 7 * 7 * 512

class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        # Block1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.norm1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.drop1 = nn.Dropout(p=0.5)
        # Block2
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.norm2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.drop2 = nn.Dropout(p=0.5)
        # Block3
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1, stride=1, padding=0)
        self.norm3 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.drop3 = nn.Dropout(p=0.5)
        # Block4
        self.conv8 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv9 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv10 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1, stride=1, padding=0)
        self.norm4 = nn.BatchNorm2d(512)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.drop4 = nn.Dropout(p=0.5)
        # Block5
        self.conv11 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv12 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv13 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1, stride=1, padding=0)
        self.norm5 = nn.BatchNorm2d(512)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.drop5 = nn.Dropout(p=0.5)
        # fc
        self.fc1 = nn.Linear(in_features=flatten_features, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=4096)
        self.drop6 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(in_features=4096, out_features=200)        

    def forward(self, x):
        # Block1
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.norm1(x)
        x = self.pool1(x)
        x = self.drop1(x)
        # Block2
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.norm2(x)
        x = self.pool2(x)
        x = self.drop2(x)
        # Block3
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.norm3(x)
        x = self.pool3(x)
        x = self.drop3(x)
        # Block4
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))
        x = self.norm4(x)
        x = self.pool4(x)
        x = self.drop4(x)
        # Block5
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.relu(self.conv13(x))
        x = self.norm5(x)
        x = self.pool5(x)
        x = self.drop5(x)
        # fc
        x = x.view(-1, flatten_features)
        #x = x.view(x.size(0),-1)
        #x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.drop6(x)
        x = F.relu(self.fc2(x))
        x = F.softmax(input=self.fc3(x), dim=0)
        return x


net = VGG16()
net = net.to(device)
#net = torch.nn.DataParallel(net)
#net.cuda()

In [ ]:
# Optimazerの設定

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# 学習

In [ ]:
print(len(trainloader))

epoch_count = 10
check_point = 100

net.train()
for epoch in range(epoch_count):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        #inputs = torch.reshape(inputs, (-1, 3, image_size, image_size))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #scheduler.setp()

        # print statistics
        running_loss += loss.item()
        if i % check_point == 0:    # print every 2000 mini-batches
            print(datetime.datetime.now())
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
# テストデータによる検証

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
#plt.imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
# 学習結果を保存

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)